# **Benchmarking DeBERTa-v3-large-squad2 fine tuned on hotpotQA for full answer (4 Layer Unfreeze)**

In [1]:
!pip install transformers datasets evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 36.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pyli

In [2]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

model_path = "/kaggle/input/deberta-v3-large-squad2-deepset-hotpot-fullanswer/kaggle/working/deberta-v3-large-squad-v2-hotpot"  # e.g., "./checkpoints/deberta-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    handle_impossible_answer=True    
)



2025-11-11 13:36:06.798744: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762868166.986784      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762868167.043670      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Device set to use cuda:0


In [3]:
from datasets import load_dataset

squad_dataset = load_dataset("squad_v2")
squad_val = squad_dataset["validation"]


README.md: 0.00B [00:00, ?B/s]

squad_v2/train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

squad_v2/validation-00000-of-00001.parqu(…):   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [4]:
predictions = []
references = []

for example in squad_val:
    context = example["context"]
    question = example["question"]
    id_ = example["id"]
    
    try:
        pred = qa_pipeline({
            "context": context,
            "question": question
        }, handle_impossible_answer=True)  # needed for squad_v2
        predicted_answer = pred.get("answer", "").strip()
    except:
        predicted_answer = ""

    predictions.append({
        "id": id_,
        "prediction_text": predicted_answer,
        "no_answer_probability": 0.0  # required but not used
    })

    references.append({
        "id": id_,
        "answers": example["answers"]
    })


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [5]:
import evaluate

metric = evaluate.load("squad_v2")
results = metric.compute(predictions=predictions, references=references)

print(f"Exact Match (EM): {results['exact']:.2f}")
print(f"F1 Score: {results['f1']:.2f}")


Exact Match (EM): 21.28
F1 Score: 40.96


# **Benchmarking DeBERTa-v3-large-squad2 fine tuned on hotpotQA for gold standard Paragraph**

In [1]:
!pip install transformers datasets evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 39.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pyli

In [6]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

model_path = "/kaggle/input/deberta-v3-large-squad2-finetune-hotpot-goldpara/debarta-squad-hotpot-final/checkpoint-2000"  # e.g., "./checkpoints/deberta-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    handle_impossible_answer=True    
)


Device set to use cuda:0


In [7]:
from datasets import load_dataset

squad_dataset = load_dataset("squad_v2")
squad_val = squad_dataset["validation"]


In [8]:
predictions = []
references = []

for example in squad_val:
    context = example["context"]
    question = example["question"]
    id_ = example["id"]
    
    try:
        pred = qa_pipeline({
            "context": context,
            "question": question
        }, handle_impossible_answer=True)  # needed for squad_v2
        predicted_answer = pred.get("answer", "").strip()
    except:
        predicted_answer = ""

    predictions.append({
        "id": id_,
        "prediction_text": predicted_answer,
        "no_answer_probability": 0.0  # required but not used
    })

    references.append({
        "id": id_,
        "answers": example["answers"]
    })


In [9]:
import evaluate

metric = evaluate.load("squad_v2")
results = metric.compute(predictions=predictions, references=references)

print(f"Exact Match (EM): {results['exact']:.2f}")
print(f"F1 Score: {results['f1']:.2f}")


Exact Match (EM): 25.65
F1 Score: 44.65


# **Benchmarking DeBERTa-v3-large fine tune on SQuAD 2.0 (3/4)**

**For only .pt file model**

In [10]:
import torch
from transformers import AutoTokenizer, DebertaV2ForQuestionAnswering

# Load saved state dict
state = torch.load("/kaggle/input/deberta-v3-large-finetune-squad2-0/step24648_epoch0.pt")

# Load base model and tokenizer
model = DebertaV2ForQuestionAnswering.from_pretrained("microsoft/deberta-v3-large")
model.load_state_dict(state["model"], strict=False)

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")

qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    handle_impossible_answer=True    
)


config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/874M [00:00<?, ?B/s]

Some weights of DebertaV2ForQuestionAnswering were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0


In [11]:
from datasets import load_dataset

squad_dataset = load_dataset("squad_v2")
squad_val = squad_dataset["validation"]


In [12]:
predictions1 = []
references1 = []

for example in squad_val:
    context = example["context"]
    question = example["question"]
    id_ = example["id"]
    
    try:
        pred = qa_pipeline({
            "context": context,
            "question": question
        }, handle_impossible_answer=True)  # needed for squad_v2
        predicted_answer = pred.get("answer", "").strip()
    except:
        predicted_answer = ""

    predictions1.append({
        "id": id_,
        "prediction_text": predicted_answer,
        "no_answer_probability": 0.0  # required but not used
    })

    references1.append({
        "id": id_,
        "answers": example["answers"]
    })


In [13]:
import evaluate

metric1 = evaluate.load("squad_v2")
results1 = metric1.compute(predictions=predictions1, references=references1)

print(f"Exact Match (EM): {results1['exact']:.2f}")
print(f"F1 Score: {results1['f1']:.2f}")


Exact Match (EM): 84.44
F1 Score: 87.98


# **DeBERTa-v3-large-squad2**

In [2]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

model_path = "deepset/deberta-v3-large-squad2"  # e.g., "./checkpoints/deberta-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    handle_impossible_answer=True    
)


2025-11-11 17:20:24.262328: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762881624.449649      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762881624.525083      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

Device set to use cuda:0


In [3]:
from datasets import load_dataset

squad_dataset = load_dataset("squad_v2")
squad_val = squad_dataset["validation"]


README.md: 0.00B [00:00, ?B/s]

squad_v2/train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

squad_v2/validation-00000-of-00001.parqu(…):   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [4]:
predictions = []
references = []

for example in squad_val:
    context = example["context"]
    question = example["question"]
    id_ = example["id"]
    
    try:
        pred = qa_pipeline({
            "context": context,
            "question": question
        }, handle_impossible_answer=True)  # needed for squad_v2
        predicted_answer = pred.get("answer", "").strip()
    except:
        predicted_answer = ""

    predictions.append({
        "id": id_,
        "prediction_text": predicted_answer,
        "no_answer_probability": 0.0  # required but not used
    })

    references.append({
        "id": id_,
        "answers": example["answers"]
    })


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [5]:
import evaluate

metric = evaluate.load("squad_v2")
results = metric.compute(predictions=predictions, references=references)

print(f"Exact Match (EM): {results['exact']:.2f}")
print(f"F1 Score: {results['f1']:.2f}")


Exact Match (EM): 87.21
F1 Score: 90.61


# **BigBird-RoBERTa-large fine tuned on MASHQA**

In [6]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

model_path = "/kaggle/input/bigbird-finetuned-mashqa/bigbird-roberta-large-mashqa"  # e.g., "./checkpoints/deberta-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    handle_impossible_answer=True    
)


Device set to use cuda:0


In [7]:
from datasets import load_dataset

squad_dataset = load_dataset("squad_v2")
squad_val = squad_dataset["validation"]


In [8]:
predictions = []
references = []

for example in squad_val:
    context = example["context"]
    question = example["question"]
    id_ = example["id"]
    
    try:
        pred = qa_pipeline({
            "context": context,
            "question": question
        }, handle_impossible_answer=True)  # needed for squad_v2
        predicted_answer = pred.get("answer", "").strip()
    except:
        predicted_answer = ""

    predictions.append({
        "id": id_,
        "prediction_text": predicted_answer,
        "no_answer_probability": 0.0  # required but not used
    })

    references.append({
        "id": id_,
        "answers": example["answers"]
    })


Attention type 'block_sparse' is not possible if sequence_length: 172 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


In [9]:
import evaluate

metric1 = evaluate.load("squad_v2")
results1 = metric1.compute(predictions=predictions, references=references)

print(f"Exact Match (EM): {results1['exact']:.2f}")
print(f"F1 Score: {results1['f1']:.2f}")


Exact Match (EM): 21.43
F1 Score: 22.51


# **BigBird-RoBERTa-large**

In [10]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

model_path = "google/bigbird-roberta-large"  # e.g., "./checkpoints/deberta-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    handle_impossible_answer=True    
)


tokenizer_config.json:   0%|          | 0.00/969 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/846k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

Some weights of BigBirdForQuestionAnswering were not initialized from the model checkpoint at google/bigbird-roberta-large and are newly initialized: ['qa_classifier.intermediate.dense.bias', 'qa_classifier.intermediate.dense.weight', 'qa_classifier.output.LayerNorm.bias', 'qa_classifier.output.LayerNorm.weight', 'qa_classifier.output.dense.bias', 'qa_classifier.output.dense.weight', 'qa_classifier.qa_outputs.bias', 'qa_classifier.qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


In [11]:
from datasets import load_dataset

squad_dataset = load_dataset("squad_v2")
squad_val = squad_dataset["validation"]


In [12]:
predictions = []
references = []

for example in squad_val:
    context = example["context"]
    question = example["question"]
    id_ = example["id"]
    
    try:
        pred = qa_pipeline({
            "context": context,
            "question": question
        }, handle_impossible_answer=True)  # needed for squad_v2
        predicted_answer = pred.get("answer", "").strip()
    except:
        predicted_answer = ""

    predictions.append({
        "id": id_,
        "prediction_text": predicted_answer,
        "no_answer_probability": 0.0  # required but not used
    })

    references.append({
        "id": id_,
        "answers": example["answers"]
    })


Attention type 'block_sparse' is not possible if sequence_length: 172 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


In [13]:
import evaluate

metric1 = evaluate.load("squad_v2")
results1 = metric1.compute(predictions=predictions, references=references)

print(f"Exact Match (EM): {results1['exact']:.2f}")
print(f"F1 Score: {results1['f1']:.2f}")


Exact Match (EM): 0.08
F1 Score: 3.20


# **DeBERTa-v3-large fine tuned on Squad2.0 for 2 Epoch**

In [3]:
import torch
from transformers import AutoTokenizer, DebertaV2ForQuestionAnswering, pipeline

# Load base model and tokenizer
model = DebertaV2ForQuestionAnswering.from_pretrained("microsoft/deberta-v3-large")

# Load fine-tuned weights (directly)
state_dict = torch.load("/kaggle/input/deberta-v3-large-squad2-2epoch/best_model.pt", map_location="cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(state_dict, strict=False)

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")

# Move to GPU if available
model.to("cuda" if torch.cuda.is_available() else "cpu")

# Create QA pipeline
qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    handle_impossible_answer=True,
    device=0 if torch.cuda.is_available() else -1
)

print("✅ Model and tokenizer loaded successfully!")


Some weights of DebertaV2ForQuestionAnswering were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0


✅ Model and tokenizer loaded successfully!


In [4]:
from datasets import load_dataset

squad_dataset = load_dataset("squad_v2")
squad_val = squad_dataset["validation"]


README.md: 0.00B [00:00, ?B/s]

squad_v2/train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

squad_v2/validation-00000-of-00001.parqu(…):   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [6]:
predictions1 = []
references1 = []

for example in squad_val:
    context = example["context"]
    question = example["question"]
    id_ = example["id"]
    
    try:
        pred = qa_pipeline({
            "context": context,
            "question": question
        }, handle_impossible_answer=True)  # needed for squad_v2
        predicted_answer = pred.get("answer", "").strip()
    except:
        predicted_answer = ""

    predictions1.append({
        "id": id_,
        "prediction_text": predicted_answer,
        "no_answer_probability": 0.0  # required but not used
    })

    references1.append({
        "id": id_,
        "answers": example["answers"]
    })


In [7]:
import evaluate

metric1 = evaluate.load("squad_v2")
results1 = metric1.compute(predictions=predictions1, references=references1)

print(f"Exact Match (EM): {results1['exact']:.2f}")
print(f"F1 Score: {results1['f1']:.2f}")


Exact Match (EM): 85.81
F1 Score: 89.06
